### FLiBe Simple

In [1]:
import openmc
import numpy as np
import matplotlib.pyplot as plt

### Materials

In [23]:
# User specifications
DENSITY_FLIBE = 1.8 # usu 1.8-2.0 g/cc
ENRICH_LI = 20
MOL_LIF, MOL_BEF2 = 0.66, 0.34

# Materials definitions
#   OpenMC automatically normalizes the fraction of each element in material (like MCNP)
flibe = openmc.Material(name="FLiBe",temperature=900)
flibe.set_density('g/cm3', DENSITY_FLIBE) # usu 1.8-2.0 g/cc
flibe.add_element('Li',MOL_LIF,'ao',ENRICH_LI,'Li6','wo')
flibe.add_element('Be',MOL_BEF2,'ao')
flibe.add_element('F',(MOL_LIF+2*MOL_BEF2),'ao')

uranium = openmc.Material(name="U",temperature=900)
# uranium.add_element('U',100,'ao',0.71)
uranium.add_nuclide('U238',100)

volume = 342 * 100e3 # cm3
M_U = 0.0181 * 20/50
M_FLiBe = 1

molratio_U, molratio_FLiBe = M_U / (M_U + M_FLiBe), M_FLiBe / (M_U + M_FLiBe)

mix = openmc.Material.mix_materials([flibe, uranium], [molratio_FLiBe, molratio_U], 'ao')

# flibe.add_s_alpha_beta() # omitted in initial minimal working example (MWE)

print(mix.get_nuclide_atom_densities()) # Returns one or all elements in the material and their atomic densities in units of atom/b-cm
print(f"Patrick's hand calcs for 1.8 g/cc, 0.64 mol LiF, 0.33 mol BeF2:")
print(f"  'Li6': 0.00489411249582549, 'Li7': 0.016783401745946593, 'Be9': 0.011167204306367438, 'F19': 0.04401192285450695")

materials = openmc.Materials([mix])
materials.cross_sections = '/mnt/c/OpenMC/data/endfb-viii.0-hdf5/cross_sections.xml'
materials.export_to_xml("materials.xml")

{'Li6': np.float64(0.004893996274411257), 'Li7': np.float64(0.01678333785152389), 'Be9': np.float64(0.011167111519421137), 'F19': np.float64(0.04401155716477743), 'U238': np.float64(0.000556437460345368)}
Patrick's hand calcs for 1.8 g/cc, 0.64 mol LiF, 0.33 mol BeF2:
  'Li6': 0.00489411249582549, 'Li7': 0.016783401745946593, 'Be9': 0.011167204306367438, 'F19': 0.04401192285450695


#### Geometry
For our simple case suppose we make a 1 cm$^3$ cube with reflective surfaces of homogeneous FLiBe.

In [24]:
# Geometry definitions
xmin = openmc.XPlane(-0.5, boundary_type='reflective')
xmax = openmc.XPlane( 0.5, boundary_type='reflective')
ymin = openmc.YPlane(-0.5, boundary_type='reflective')
ymax = openmc.YPlane( 0.5, boundary_type='reflective')
zmin = openmc.ZPlane(-0.5, boundary_type='reflective')
zmax = openmc.ZPlane( 0.5, boundary_type='reflective')

region = +xmin & -xmax & +ymin & -ymax & +zmin & -zmax
cell = openmc.Cell(fill=mix, region=region)
root_univ = openmc.Universe(cells=[cell])

geometry = openmc.Geometry(root_univ)
geometry.export_to_xml("geometry.xml")

#### Settings

In [25]:
settings = openmc.Settings()

# Tallies
tallies = openmc.Tallies()
tally = openmc.Tally(name='capture')
tally.scores = ['(n,gamma)']
tally.nuclides = ['U238']
tallies.append(tally)
tallies.export_to_xml()

# Uniform source in cube
# --Isotropic 14 MeV volumetric source throughout cube (idk)
source = openmc.IndependentSource()
source.space = openmc.stats.Box((-0.5, -0.5, -0.5), (0.5, 0.5, 0.5))
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([14.0e6], [1.0])
# source.time = openmc.stats.Uniform(0, 1e-6)
settings.source = source

# Run type
settings.run_mode = 'fixed source'
settings.particles = 100000
settings.generations_per_batch = 10
settings.batches = 100
settings.inactive = 5

settings.export_to_xml()

Run OpenMC

In [26]:
# if __name__ == '__main__':
#     # If you were to make this a .py script then you should place everything below underneath this if statement

openmc.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [19]:
sp = openmc.StatePoint('statepoint.100.h5')
tally = sp.get_tally(scores=['(n,gamma)'])

LookupError: Unable to get Tally